In [1]:
import gc, functools,time
import pickle, re
import random, math
from typing import List
from pprint import pprint

In [2]:
from pyecharts.charts import Bar, Tab, Grid, Line
from pyecharts import options as opts
from pyecharts.globals import ThemeType

In [18]:


import  pandas as pd
import nltk
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from pyLDAvis.sklearn import prepare
import pyLDAvis
import pyLDAvis.gensim

In [10]:
def read_text(road):
    filetxtlist = []
    with open(road, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            line = line.strip('\n').replace(u'\u3000',u' ')
            filetxtlist.append(line)
    filetxt = ''.join(filetxtlist)
    return filetxt

road = '../data/df.txt'
t = read_text(road)
print(t[0:300])

.x月xx日推出凭证式国债x年期x.xx.xx%，x年期x.xx%到期一次还本付息。真情邮政，为您竭诚服务！  咨询电话xxxx-xxx强度等级水泥的必要性和可行性进行深入研究Don’tSellaProduct以上比赛规则由江苏科技大学教职工摄影协会负责解释坐12个小时飞机身体已经疲惫不堪为什么不能是你③以多数人的努力程度地址位于天津市滨海新区响罗湾旷世国际大厦A座1801室它是由AlexanderStepanov、MengLee和DavidRMusser在惠普实验室工作时所开发出来的前首席执行官迪克·科斯特洛或将离开zuzu气垫BB拍上去过几分钟后就会和皮肤越来越贴年薪20万以上的工作岗位普


In [12]:
roads = ['../data/message.txt']
filetxt = []
for f in roads:
    filetxt.append(read_text(f))

In [14]:
# 文本清理
def clearn_text(text):
    '''新增功能：删除英文、数字
    # 版本一：过滤filter_key：result为[0,0,1,0]
    # 版本二：过滤英文、数字：result为[1, 0, 1, 1]
    输入数据的质量和结果输出紧密相关
    '''
    # filter_key = ['0','1','2','3','4','5','6','7','8','9','０','１','２','３','４','５','６','７','８','９']
    # for keys in filter_key:
    #     text = text.replace(keys,'')
    new_text = re.sub('[a-zA-Z0-9]','',text)
    return new_text
filetxt = [clearn_text(text) for text in filetxt]
#filetxt

## 加载停用词


In [4]:

print(len(stopwords))

746


In [16]:
# 中文停用词载入
# nltk.download('stopwords')
# stopwords = nltk.corpus.stopwords.words('chinese')

# 停用词过滤
def clean_text(text):
    '''
    完善功能：无关词汇能过滤就过滤掉，否则影响结果
    # 版本一：stopwords比较少时：result为[1, 0, 1, 1]
    # 版本二：丰富stopwords词汇，过滤无关字符：result为[1, 1, 0, 0]（是理想结果）
    输入数据的质量和结果输出紧密相关
    '''
#     stopwords = ['的','了','但是','况且','不仅','还','为了','主要','正文','作者','关键词','称之为',
#     '认为','因此','人们','一个','需要','我们','你们','他们','这种','标题','进一步','如图所示',
#     '一种','一定','由于','条件','进行','作为','一个','可以','mm','原文','更进一步','文献号',
#     'xi','结果','版号','标题','日期','合作','去年','今年','目前','期号','作者简介','哲社版',
#     '昨天','一直','作者','存在','正文','形成','要求','产生','因为','因此','期号']
    stopwords = open('../data/cn_stopwords.txt',encoding='utf-8').read().split()
    stopwords = frozenset(stopwords)
    wordlist = jieba.lcut(text) 
    # 去除停用词和长度小于2的词语
    wordlist = [w for w in wordlist if w not in stopwords and len(w)>2]
    # 词语之间以空格间隔
    document = ' '.join(wordlist)
    return document
corpus = [clean_text(text) for text in filetxt]
#corpus

In [19]:

vectorizer = CountVectorizer()
doc_term_matrix = vectorizer.fit_transform(corpus)  # 输入：list格式
matrix_features = vectorizer.get_feature_names()
matrix_dict = sorted(vectorizer.vocabulary_.items(), key=lambda item: item[1], reverse=False)
print('给每个词一个编号：{}...'.format(str(matrix_dict)[0:56]))
print('涉及的特征：{}，...'.format('，'.join(matrix_features[0:4])))
print('涉及特征数：{}'.format(len(matrix_features)))

给每个词一个编号：[('一个半', 0), ('一个多', 1), ('一个打', 2), ('一个月', 3), ('一会儿',...
涉及的特征：一个半，一个多，一个打，一个月，...
涉及特征数：942


In [28]:
from sklearn.decomposition import LatentDirichletAllocation
sklearn_lda = LatentDirichletAllocation(n_components=100, 
                                        doc_topic_prior=None,
                                        topic_word_prior=None, 
                                        learning_method='batch', 
                                        learning_decay=0.7, 
                                        learning_offset=10.0, 
                                        max_iter=50,
                                        batch_size=256,
                                        evaluate_every=-1, 
                                        total_samples=1000000.0, 
                                        perp_tol=0.1, 
                                        mean_change_tol=0.001, 
                                        max_doc_update_iter=100, 
                                        n_jobs=-1, 
                                        verbose=2, 
                                        random_state=None)

sklearn_lda.fit(doc_term_matrix)
tf_feature_names = vectorizer.get_feature_names()
#print_top_words(lda_model , tf_feature_names, n_top_words=20)

# 最小困惑度所对应的Topic就是最优的主题数 
#print(u'\n困惑度：',lda_model.perplexity(doc_term_matrix,sub_sampling = False))  # doc_term_matrix 特征矩阵

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:   

iteration: 1 of max_iter: 50
iteration: 2 of max_iter: 50
iteration: 3 of max_iter: 50
iteration: 4 of max_iter: 50
iteration: 5 of max_iter: 50
iteration: 6 of max_iter: 50
iteration: 7 of max_iter: 50
iteration: 8 of max_iter: 50
iteration: 9 of max_iter: 50
iteration: 10 of max_iter: 50
iteration: 11 of max_iter: 50
iteration: 12 of max_iter: 50
iteration: 13 of max_iter: 50
iteration: 14 of max_iter: 50
iteration: 15 of max_iter: 50
iteration: 16 of max_iter: 50
iteration: 17 of max_iter: 50
iteration: 18 of max_iter: 50


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:   

iteration: 19 of max_iter: 50
iteration: 20 of max_iter: 50
iteration: 21 of max_iter: 50
iteration: 22 of max_iter: 50
iteration: 23 of max_iter: 50
iteration: 24 of max_iter: 50
iteration: 25 of max_iter: 50
iteration: 26 of max_iter: 50
iteration: 27 of max_iter: 50
iteration: 28 of max_iter: 50
iteration: 29 of max_iter: 50
iteration: 30 of max_iter: 50
iteration: 31 of max_iter: 50
iteration: 32 of max_iter: 50
iteration: 33 of max_iter: 50
iteration: 34 of max_iter: 50
iteration: 35 of max_iter: 50
iteration: 36 of max_iter: 50
iteration: 37 of max_iter: 50
iteration: 38 of max_iter: 50
iteration: 39 of max_iter: 50
iteration: 40 of max_iter: 50
iteration: 41 of max_iter: 50
iteration: 42 of max_iter: 50
iteration: 43 of max_iter: 50
iteration: 44 of max_iter: 50
iteration: 45 of max_iter: 50
iteration: 46 of max_iter: 50
iteration: 47 of max_iter: 50
iteration: 48 of max_iter: 50
iteration: 49 of max_iter: 50
iteration: 50 of max_iter: 50


[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concu

In [29]:
sklearn_lda.components_

array([[0.01, 0.01, 0.01, ..., 0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01, ..., 0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01, ..., 0.01, 0.01, 0.01],
       ...,
       [0.01, 0.01, 0.01, ..., 0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01, ..., 0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01, ..., 0.01, 0.01, 0.01]])

In [30]:
lda_representation  = sklearn_lda.transform(doc_term_matrix)  # get topics for some given samples:

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    0.0s finished


In [31]:
def print_top_words(model, feature_names, n_top_words=10):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ",".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [32]:
print("\nTopics in LDA model:")
tf_feature_names = vectorizer.get_feature_names()
print_top_words(sklearn_lda , tf_feature_names, n_top_words=20)


Topics in LDA model:
Topic #0: 鼓楼区,声嘶力竭,大方向,大放送,大干一场,大崗山,大峡谷,大学生,大姑娘,大吐苦水,多美元,多渠道,夏迪特,处方药,墨尔本,地理位置,基础设施,基督教,培训基地,城管局
Topic #1: 鼓楼区,声嘶力竭,大方向,大放送,大干一场,大崗山,大峡谷,大学生,大姑娘,大吐苦水,多美元,多渠道,夏迪特,处方药,墨尔本,地理位置,基础设施,基督教,培训基地,城管局
Topic #2: 鼓楼区,声嘶力竭,大方向,大放送,大干一场,大崗山,大峡谷,大学生,大姑娘,大吐苦水,多美元,多渠道,夏迪特,处方药,墨尔本,地理位置,基础设施,基督教,培训基地,城管局
Topic #3: 鼓楼区,声嘶力竭,大方向,大放送,大干一场,大崗山,大峡谷,大学生,大姑娘,大吐苦水,多美元,多渠道,夏迪特,处方药,墨尔本,地理位置,基础设施,基督教,培训基地,城管局
Topic #4: 鼓楼区,声嘶力竭,大方向,大放送,大干一场,大崗山,大峡谷,大学生,大姑娘,大吐苦水,多美元,多渠道,夏迪特,处方药,墨尔本,地理位置,基础设施,基督教,培训基地,城管局
Topic #5: 鼓楼区,声嘶力竭,大方向,大放送,大干一场,大崗山,大峡谷,大学生,大姑娘,大吐苦水,多美元,多渠道,夏迪特,处方药,墨尔本,地理位置,基础设施,基督教,培训基地,城管局
Topic #6: 鼓楼区,声嘶力竭,大方向,大放送,大干一场,大崗山,大峡谷,大学生,大姑娘,大吐苦水,多美元,多渠道,夏迪特,处方药,墨尔本,地理位置,基础设施,基督教,培训基地,城管局
Topic #7: 鼓楼区,声嘶力竭,大方向,大放送,大干一场,大崗山,大峡谷,大学生,大姑娘,大吐苦水,多美元,多渠道,夏迪特,处方药,墨尔本,地理位置,基础设施,基督教,培训基地,城管局
Topic #8: 鼓楼区,声嘶力竭,大方向,大放送,大干一场,大崗山,大峡谷,大学生,大姑娘,大吐苦水,多美元,多渠道,夏迪特,处方药,墨尔本,地理位置,基础设施,基督教,培训基地,城管局
Topic #9: 鼓楼区,声嘶力竭,大方向,大放送,大干一场,大崗山,大峡谷,大学生,大姑娘,大吐苦水,多美元,多渠道,夏迪特,处方药,墨尔本,地理位置,基础设施,基督教,培训基地,城管局
Topic #10: 鼓楼区,声嘶力